# Khám phá dữ liệu
**Checklist:**
1. Tổng quan về tập dữ liệu
2. Phân tích Cột Số
3. Phân tích các cột phân loại
4. Phân tích các cột tọa độ
5. Phân tích cột thời gian
6. Phân tích dữ liệu thiếu
7. Mối quan hệ và Tương quan
8. Quan sát ban đầu và Insight

## Import các thư viện cần thiết

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # setup đường dẫn

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns 

## Load dữ liệu

In [ ]:
input_path = "../data/raw/Global_Landslide_Catalog_Export.csv"
df = pd.read_csv(input_path)

## 1. Tổng quan về tập dữ liệu

### Thông tin cơ bản

In [ ]:
df.head()

In [ ]:
df.tail()

#### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [ ]:
print("Số dòng:", df.shape[0])
print("Số cột:", df.shape[1])

#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
Mỗi dòng ứng với một sự kiện sạt lở đất. Các dòng đều có ý nghĩa giống nhau.

#### Kích thước tổng thể của tập dữ liệu là bao nhiêu?

In [ ]:
total_usage_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)

print(f"Dung lượng bộ nhớ: {total_usage_mb:.2f} MB")

### Tính Toàn vẹn Dữ liệu

#### Có các dòng bị trùng lặp không? Nếu có, số lượng là bao nhiêu?

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated('event_id').sum()

-> Không có dòng nào bị trùng lặp, trùng event_id

#### Có cần phải xóa các dòng bị trùng không?

-> Do không có dòng nào bị trùng lặp nên không cần phải xóa

#### Tất cả các dòng đã đầy đủ thông tin chưa, hay một số dòng hoàn toàn trống?

In [ ]:
rows_completely_empty = df.isna().all(axis=1)
count_empty = rows_completely_empty.sum()
print(f"Số hàng hoàn toàn trống: {count_empty}")

# Xem các dòng đó (nếu có)
if count_empty:
    df[rows_completely_empty]

### Danh mục cột

#### Ý nghĩa/định nghĩa của từng cột là gì?

| Tên thuộc tính (Feature) | Ý nghĩa/định nghĩa |
| :--- | :--- |
| **source_name** | Tên của nguồn tin báo cáo về sự kiện sạt lở |
| **source_link** | Đường dẫn (Link) đến nguồn gốc của báo cáo |
| **event_id** | Mã định danh duy nhất cho sự kiện sạt lở |
| **event_date** | Ngày xảy ra sự kiện sạt lở |
| **event_time** | Giờ xảy ra sự kiện (lưu ý: trường này có thể thiếu nhiều giá trị) |
| **event_title** | Tiêu đề hoặc tên gọi của sự kiện sạt lở |
| **event_description** | Mô tả chi tiết về sự kiện sạt lở |
| **location_description** | Mô tả về địa điểm nơi xảy ra sạt lở |
| **location_accuracy** | Độ chính xác của dữ liệu vị trí địa lý |
| **landslide_category** | Phân loại sạt lở (ví dụ: landslide, mudslide...) |
| **landslide_size** | Kích thước/quy mô của vụ sạt lở |
| **landslide_trigger** | Nguyên nhân kích hoạt vụ sạt lở (ví dụ: monsoon, rain, downpour,...) |
| **landslide_setting** | Bối cảnh địa lý/môi trường xảy ra |
| **fatality_count** | Số lượng người tử vong do sạt lở |
| **injury_count** | Số lượng người bị thương do sạt lở |
| **storm_name** | Tên cơn bão (nếu vụ sạt lở do bão gây ra) |
| **photo_link** | Đường dẫn đến hình ảnh minh họa vụ sạt lở |
| **notes** | Các ghi chú hoặc nhận xét bổ sung về sự kiện |
| **event_import_source** | Nguồn gốc nơi dữ liệu sự kiện được nhập vào hệ thống |
| **event_import_id** | Mã ID của sự kiện từ nguồn nhập liệu |
| **country_name** | Tên quốc gia nơi xảy ra sạt lở |
| **country_code** | Mã quốc gia |
| **admin_division_name** | Tên đơn vị hành chính (ví dụ: tên Tỉnh hoặc Bang) nơi xảy ra sạt lở |
| **admin_division_population** | Dân số của đơn vị hành chính đó |
| **gazeteer_closest_point** | Tên địa danh hoặc điểm mốc địa lý gần nhất với vị trí sạt lở |
| **gazeteer_distance** | Khoảng cách từ vị trí sạt lở đến điểm mốc địa lý gần nhất |
| **submitted_date** | Ngày sự kiện được gửi vào danh mục dữ liệu |
| **created_date** | Ngày sự kiện được tạo trong danh mục dữ liệu |
| **last_edited_date** | Ngày dữ liệu về sự kiện được chỉnh sửa lần cuối |
| **longitude** | Kinh độ của vị trí sạt lở |
| **latitude** | Vĩ độ của vị trí sạt lở |

#### Có cột nào nên được loại bỏ không? Tại sao?

In [ ]:
# Xác định số lượng giá trị thiếu và tỷ lệ của các feature
df_cols = pd.DataFrame({'Count Missing': df.isna().sum(),
                        'Percent Missing': df.isnull().sum()*100/df.shape[0]})

df_cols = df_cols[df_cols['Percent Missing'] > 80]

print("Các cột thiếu hơn 80% dữ liệu")
df_cols

Nhóm quyết định loại bỏ các cột sau khỏi bộ dữ liệu để tập trung vào việc phân tích các yếu tố ảnh hưởng đến sạt lở:
- Loại bỏ Metadata hệ thống: Các cột như `event_import_id, event_import_source, submitted_date, created_date, last_edited_date` được loại bỏ vì chúng chỉ phản ánh quy trình nhập liệu hành chính, không mang thông tin về đặc điểm vật lý của sự kiện sạt lở.
- Loại bỏ thông tin dư thừa: Cột `country_code` bị loại bỏ vì đã có `country_name`.
- Loại bỏ dữ liệu không cấu trúc: Các cột `source_link và photo_link` được loại bỏ vì chúng là các đường dẫn web, không phục vụ cho việc phân tích thống kê định lượng.
- Các cột có dữ liệu thiếu quá nhiều (dựa vào phân tích phía trên): Các cột `event_time, storm_name, notes` đều có **Percent Missing** > 80%.

#### Những cột nào có liên quan đến việc phân tích tiềm năng?

Theo như ý nghĩa/định nghĩa của từng cột, dữ liệu có trong các cột và đã xác định các cột cần loại bỏ thì nhóm có tổng hợp lại danh sách các cột có ý nghĩa phân tích cao của dataset như sau:
1. Phân tích mức độ thiệt hại về người
- `fatality_count`: Số người tử vong. Dùng để phân tích mức độ nguy hiểm chết người.
- `injury_count`: Số người bị thương. Dùng để đánh giá tác động sức khỏe cộng đồng.
2. Phân tích đặc điểm vụ sạt lở
- `landslide_category`: Giúp phân loại đặc tính rủi ro của từng vùng địa hình.
- `landslide_trigger`: Tìm mối tương quan giữa nguyên nhân và số lượng thương vong
- `landslide_size`: Dùng để phân tích tương quan giữa quy mô vật lý và thiệt hại con người.
- `landslide_setting`: Kiểm tra khu vực nào dễ bị sạt lở
3. Phân tích không gian – địa lý
- `latitude và longitude`: Hai biến số quan trọng nhất để trực quan hóa dữ liệu trên bản đồ (Heatmap) và phân tích cụm địa lý.
- `country_name và admin_division_name`: Dùng để so sánh tần suất và mức độ thiệt hại giữa các quốc gia hoặc các vùng lãnh thổ (Tỉnh/Bang).
- `admin_division_population`: Liên hệ dân số với mức độ thiệt hại
4. Phân tích thời gian
- `event_date`: Có thể phân tích nhằm xác định "mùa sạt lở" trong năm, hoặc xu hướng tăng giảm số vụ sạt lở qua các năm.

Những cột ít quan trọng nhưng vẫn hữu ích cần xử lý NLP: `location_description và event_description`

### Kiểu Dữ liệu

#### Kiểu dữ liệu hiện tại của từng cột là gì?

In [ ]:
df.dtypes

#### Có cột nào mang kiểu dữ liệu không phù hợp không? Những cột nào cần thực hiện chuyển đổi kiểu dữ liệu?

Các cột có kiểu dữ liệu không phù hợp là:
- `event_date, submitted_date, created_date, last_edited_date` đang ở kiểu dữ liệu object cần chuyển sang kiểu dữ liệu datatime.
- `event_id, event_import_id` đang ở kiểu dữ liệu int64 cần chuyển sang kiểu dữ liệu object hoặc khi xét cột số thì loại bỏ các cột này ra.

## 2. Phân tích Cột Số
Trước khi phân tích các cột số thì nhóm sẽ xử lý nhẹ kiểu dữ liệu của các cột chưa phù hợp, và xóa các cột không cần thiết

In [ ]:
# Thực hiện loại bỏ các cột không cần thiết
cols_to_drop = [
    'event_import_id', 'event_import_source', 
    'created_date', 'submitted_date', 'last_edited_date',
    'source_link', 'photo_link', 'storm_name',
    'country_code', 'event_time', 'notes',
]

df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# Xử lý cột Thời gian (Chuyển từ object sang datetime)
df['event_date'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, errors='coerce')

Dataset có các cột số là: `fatality_count, injury_count, admin_division_population, gazeteer_distance`

### Phân phối và Xu hướng tập trung

#### Hình dạng phân phối như thế nào? Vẽ biểu đồ trực quan.

In [ ]:
# Hàm format tick: chia 1.000.000 để chuyển sang đơn vị triệu
def millions(x, pos):
    return '{:,.1f}'.format(x / 1_000_000)

# Danh sách các cột cần vẽ
cols_to_plot = {
    'fatality_count': 'Number of Fatalities', 
    'injury_count': 'Number of Injuries', 
    'admin_division_population': 'Administrative Division Population', 
    'gazeteer_distance': 'Distance to Nearest Geographic Landmark'
}

n_cols = 2
n_rows = 2

plt.figure(figsize=(15, 5 * n_rows))

for i, (col, col_name) in enumerate(cols_to_plot.items()):
    # Tạo vị trí cho từng biểu đồ
    ax = plt.subplot(n_rows, n_cols, i + 1)
    
    if col == 'gazeteer_distance':
        sns.histplot(df[col].dropna(), kde=True, bins=30, color='skyblue', edgecolor='black', ax=ax)
        ax.set_xlabel(col_name + ' (km)', fontsize=12)
    elif col == 'admin_division_population':
        sns.histplot(df[col].dropna(), kde=True, bins=30, color='skyblue', edgecolor='black', ax=ax)
        ax.xaxis.set_major_formatter(FuncFormatter(millions))
        ax.set_xlabel(col_name + ' (millions)', fontsize=12)
    else:
        sns.histplot(df[col].dropna(), bins=30, color='skyblue', edgecolor='black', ax=ax)
        ax.set_xlabel(col_name, fontsize=12)
        
    # Trang trí
    ax.set_title(f'Distribution of {col_name}', fontsize=14)
    ax.set_ylabel('Number of Landslide', fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

**Nhận xét:**
- Hai biến fatality_count (số người tử vong) và injury_count (số người bị thương) có phân phối lệch phải rất mạnh. Phần lớn các vụ sạt lở không gây thiệt hại về người hoặc chỉ gây thiệt hại rất nhỏ, thể hiện qua số lượng lớn giá trị bằng 0 hoặc gần 0.
- Hai biến admin_division_population (dân số của đơn vị hành chính) và gazeteer_distance (khoảng cách từ vị trí sạt lở đến điểm mốc địa lý gần nhất) là các biến liên tục và thể hiện sự phân tán lớn trong dữ liệu, có phân phối lệch phải.

#### Thống kê mô tả

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
cols_to_analyze = ['fatality_count', 'injury_count', 'admin_division_population', 'gazeteer_distance']
print("--- THỐNG KÊ MÔ TẢ CƠ BẢN ---")
df[cols_to_analyze].describe()

**Nhận xét:**
1. Nhóm biến về thiệt hại con người (fatality_count, injury_count):
- Trung vị (50%) = 0: Với cả số người chết và bị thương, giá trị trung vị là 0. Điều này nghĩa là hơn 50% số vụ sạt lở trong dữ liệu không gây thiệt hại về người (hoặc không được báo cáo).
- Phân vị thứ 3 (75%) của fatality là 1: Nghĩa là 75% các vụ sạt lở có số người chết nhỏ hơn hoặc bằng 1.
- fatality_count: Mean (3.22) lớn hơn nhiều so với Median (0), min = 0 và max = 5000, độ lệch chuẩn là 59.89. Biến này bị lệch phải cực nặng.
- injury_count: Mean (0.75) lớn hơn so với Median (0), min = 0 và max = 374, độ lệch chuẩn là 8.46. Biến này cũng bị lệch phải.

2. Biến dân số (admin_division_population):
- min = 0, max = 12691836
- Mean (157k) lớn hơn nhiểu so với Median (7.3k). Biến này bị lệch phải cực nặng.

3. Biến khoảng cách (gazeteer_distance):
- Median (6.25 km): 50% các vụ sạt lở nằm trong bán kính khoảng 6km so với một địa danh.
- min = 0, max = 215 km cho thấy sạt lở có thể xảy ra ngay tại một địa danh hoặc rất xa địa danh.

-> Các giá trị min/max đều hợp lý và phản ánh đúng thực tế khắc nghiệt của thiên tai

### Giá trị ngoại lai

In [ ]:
plt.figure(figsize=(12, 5)) 

# --- Biểu đồ 1: Population ---
plt.subplot(1, 2, 1)
ax1 = sns.boxplot(x=df['admin_division_population'], color='skyblue') 
plt.title('Distribution of Population')
plt.xlabel('Population (Millions)')
# Áp dụng formatter cho trục X
ax1.xaxis.set_major_formatter(FuncFormatter(millions))
plt.grid(axis='x', linestyle='--', alpha=0.7) 

# --- Biểu đồ 2: Distance ---
plt.subplot(1, 2, 2)
sns.boxplot(x=df['gazeteer_distance'], color='lightgreen')
plt.title('Distribution of Distance to Nearest Landmark')
plt.xlabel('Distance (km)')
plt.grid(axis='x', linestyle='--', alpha=0.7)


plt.tight_layout()
plt.show()

**Nhận xét:**
- Biểu đồ về dân số lệch phải rất nhiều, có nhiều điểm ngoại lai.
- Biểu đồ về khoảng cách cũng bị lệch phải, nhưng mức độ nhẹ hơn so với biểu đồ dân số, có nhiều điểm ngoại lai.

-> Các giá trị ngoại lai là giá trị cực đoan thực sự, không phải lỗi nhập liệu. Thực tế dân số của một đơn vị hành chính lớn thường rất đông, sạt lở đất thường xảy ra ở vùng núi cao, vùng sâu vùng xa, nơi có địa hình hiểm trở và ít người sinh sống nên khoảng cách đến mốc địa lý gần nhất lớn là điều bình thường.

### Chất lượng dữ liệu

In [ ]:
print("Tỷ lệ phần trăm giá trị thiếu của các cột là:")
percent_missing = df[cols_to_analyze].isnull().mean().sort_values(ascending=False) * 100
percent_missing = percent_missing.apply(lambda x: f"{x:.2f}%")
print(percent_missing)

### 

## 3. Phân tích các cột phân loại
Xét các cột phân loại sau: `location_accuracy, landslide_category, landslide_trigger, landslide_size, landslide_setting, country_name`                    

### Phân phối giá trị

#### Các cột phân loại có bao nhiêu giá trị duy nhất?

In [ ]:
cols_category = ['location_accuracy', 'landslide_category', 'landslide_trigger', 'landslide_size', 'landslide_setting', 'country_name']
for col in cols_category:
    print(f"\nSố lượng giá trị duy nhất của cột \"{col}\": {df[col].nunique()}")
    print(f"Mảng các giá trị duy nhất: {df[col].unique()}")

#### Biểu đồ trực quan các cột

In [ ]:
# --- 1. landslide_category ---
plt.figure(figsize=(10, 6))

category_labels_display = [val.replace('_',' ') for val in df.landslide_category.value_counts().index]

ax = sns.countplot(
    y='landslide_category',
    data=df,
    order=df['landslide_category'].value_counts().index,
    color='teal'
)
plt.title('Number of Landslides by Category', fontsize=14)
plt.xlabel('Number of Landslides', fontsize=12)
plt.ylabel('landslide Category', fontsize=12)
ax.set_yticklabels(category_labels_display)
for p in ax.patches:
    ax.annotate(int(p.get_width()), (p.get_width(), p.get_y() + p.get_height()/2.),
                ha='left', va='center', fontsize=9)

plt.show()

**Nhận xét:**
- Nhóm sạt lở (landslide) chiếm số lượng lớn khoảng 7600 vụ. Chứng tỏ trong quá trình thu thập dữ liệu, người ghi nhận thường sử dụng thuật ngữ "sạt lở" thay vì phân loại chi tiết ra.
- Nhóm chiếm số lượng vụ đứng thứ 2 là lũ bùn (mudslide).
- Các nhóm còn lại có số lượng vụ rất ít.

In [ ]:
# --- 2. landslide_trigger ---
plt.figure(figsize=(10, 6))

trigger_labels_display = [val.replace('_',' ') for val in df.landslide_trigger.value_counts().index]

ax = sns.countplot(
    y='landslide_trigger',
    data=df,
    order=df['landslide_trigger'].value_counts().index,
    color='teal'
)
plt.title('Number of Landslides by Trigger', fontsize=14)
plt.xlabel('Number of Landslides', fontsize=12)
plt.ylabel('Landslide Trigger', fontsize=12)
ax.set_yticklabels(trigger_labels_display)
for p in ax.patches:
    ax.annotate(int(p.get_width()), (p.get_width(), p.get_y() + p.get_height()/2.),
                ha='left', va='center', fontsize=9)
    
plt.show()

**Nhận xét:**
- Mưa là nguyên nhân chính, các yếu tố liên quan đến nước và thời tiết là nguyên nhân áp đảo gây ra sạt lở như mưa lớn (downpour) khoảng hơn 4500 vụ, mưa thường (rain) khoảng 2500 vụ, mưa liên tục (continuous rain), bão nhiệt đới (tropical cyclone), gió mùa (monsoon), lũ lụt (flooding).
- Nhóm không xác định (unknown) cũng có số lượng rất cao gần 1700 vụ.
- Các nguyên nhân do con người như khai khoáng (mining), xây dựng (construction), vỡ đường ống nước (leaking pipe) có số lượng vụ không nhiều.
- Các nguyên nhân do địa chấn như động đất (earthquake), rung chấn (vibration), núi lửa (volcano) cũng có số lượng rất ít.
- Các nguyên nhân hiếm gặp như tuyết tan (snowfall snowmelt), đóng băng (freeze thaw).

In [ ]:
# --- 3. landslide_setting ---
plt.figure(figsize=(10, 6))

setting_labels_display = [val.replace('_',' ') for val in df.landslide_setting.value_counts().index]

ax = sns.countplot(
    y='landslide_setting',
    data=df,
    order=df['landslide_setting'].value_counts().index,
    color='teal'
)
plt.title('Number of Landslides by Setting', fontsize=14)
plt.xlabel('Number of Landslides', fontsize=12)
plt.ylabel('Landslide Setting', fontsize=12)
ax.set_yticklabels(setting_labels_display)
for p in ax.patches:
    ax.annotate(int(p.get_width()), (p.get_width(), p.get_y() + p.get_height()/2.),
                ha='left', va='center', fontsize=9)
    
plt.show()

**Nhận xét:**
- Nhóm không xác định (unknown) chiếm số lượng lớn với hơn 6000 vụ. Chứng tỏ có hơn một nửa số vụ sạt lở không được ghi nhận về bối cảnh môi trường xung quanh.
- Nếu bỏ qua nhóm không xác định thì bối cảnh có số vụ lớn nhất là trên đường (above road) với hơn 3000 vụ.Chứng tỏ xây dựng đường xá là một trong những yếu tố rủi ro lớn nhất dẫn đến sạt lở.
- Các bối cảnh như khu vực bị cháy rừng (burned area), sườn dốc đã được gia cố kỹ thuật (engineered slope), ven biển (above coast) có số lượng vụ được ghi nhận rất ít.

In [ ]:
accuracy_order = [
    'exact', '1km', '5km', '10km', '25km', 
    '50km', '100km', '250km', 'unknown'
]

size_order = [
    'small', 'medium', 'large', 'very_large', 
    'catastrophic', 'unknown'
]

# Dùng để hiển thị trên trục hoành
accuracy_labels_display = [
    'Exact', '1km', '5km', '10km', '25km', 
    '50km', '100km', '250km', 'Unknown'
]

size_labels_display = [
    'Small', 'Medium', 'Large', 'Very Large', 
    'Catastrophic', 'Unknown'
]

# Thiết lập biểu đồ
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# --- 4. location_accuracy ---
sns.countplot(
    ax=axes[0],
    x='location_accuracy',
    data=df,
    order=accuracy_order, 
    color='teal'
)
axes[0].set_title('Number of Landslides by Location Accuracy', fontsize=14)
axes[0].set_xlabel('Location Accuracy', fontsize=12)
axes[0].set_ylabel('Number of Landslides', fontsize=12)
axes[0].set_xticklabels(accuracy_labels_display)
for p in axes[0].patches:
    axes[0].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)
    
# --- 5. landslide_size ---
sns.countplot(
    ax=axes[1],
    x='landslide_size',
    data=df,
    order=size_order, 
    color='teal'
)
axes[1].set_title('Number of Landslides by Size', fontsize=14)
axes[1].set_xlabel('Landslide Size', fontsize=12)
axes[1].set_ylabel('Number of Landslides', fontsize=12)
axes[1].set_xticklabels(size_labels_display)
for p in axes[1].patches:
    axes[1].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

**Nhận xét:**
1. Biểu đồ "Number of Landslides by Location Accuracy":
- Phần lớn các vụ sạt lở được ghi nhận với độ chính xác vị trí tương đối tốt. Nhóm 5km chiếm số lượng cao nhất (hơn 3000 vụ), tiếp theo là nhóm 1km (hơn 2000 vụ).
- Số lượng vụ sạt lở có xu hướng giảm dần khi độ chính xác tăng lên (từ 10km đến 250km).
- Số lượng vụ sạt lở có độ chính xác (exact) là khá đáng kể (khoảng 1400 vụ). Tuy nhiên, vẫn có một lượng đáng kể các vụ sạt lở mà độ chính xác vị trí là không xác định (unknown).

2. Biểu đồ "Number of Landslides by Size"
- Nhóm trung bình (medium) chiếm số lượng vượt trội với hơn 6500 vụ sạt lở.
- Nhóm nhỏ (small) là nhóm phổ biến thứ 2 với gần 3000 vụ sạt lở.
- Số lượng vụ sạt lở có quy mô lớn (large), rất lớn (very large) và thảm họa (catastrophic) là rất ít, đặc biệt là nhóm thảm họa. Điều này phản ánh thực tế rằng các sự kiện sạt lở quy mô cực lớn thường hiếm khi xảy ra.
- Nhóm không xác định (unknown) có khoảng 850 vụ, cho thấy vẫn còn nhiều vụ chưa xác định rõ quy mô.

Insight: Theo quy luật tự nhiên, sạt lở nhỏ thường xảy ra nhiều hơn sạt lở lớn. Việc nhóm "Small" thấp hơn nhóm "Medium" gợi ý rằng có thể có "thiên kiến báo cáo" (Reporting Bias) – tức là các vụ sạt lở quá nhỏ thường bị bỏ qua không được ghi nhận vào hệ thống.

In [ ]:
# --- 6. country_name ---
plt.figure(figsize=(16, 8))

ax = sns.countplot(
    x='country_name',
    data=df,
    order=df['country_name'].value_counts().head(10).index,
    color='teal'
)
plt.title('Top 10 Countries by Number of Landslides', fontsize=14)
plt.xlabel('Country Name', fontsize=12)
plt.ylabel('Number of Landslides', fontsize=12)
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

plt.show()

**Nhận xét:**
- Mỹ là quốc gia có số lượng báo cáo sạt lở cao nhất trong tập dữ liệu (với gần 3000 vụ), trong khi đó Canada và Malaysia ghi nhận khoảng 175 vụ. Về mặt địa chất, Mỹ không hẳn là nơi xảy ra sạt lở nhiều nhất thế giới, đây là dấu hiệu điển hình của Reporting Bias vì NASA có khả năng tiếp cận và ghi nhận các báo cáo từ Mỹ sẽ tốt hơn các quốc gia khác.
- Trong 10 quốc gia thì đa số thuộc Châu Á; còn lại thuộc Bắc Mỹ (United States, Canada), Nam Mỹ (Brazil), Châu Âu (United Kingdom).

#### Sự phân phối có cân bằng hay bị mất cân bằng nghiêm trọng?
-> Các biểu đồ của các cột phân loại đều có phân phối bị mất cân bằng

### Chất lượng dữ liệu

#### Tỷ lệ phần trăm giá trị bị thiếu của các cột phân loại

In [ ]:
print("Tỷ lệ phần trăm giá trị thiếu của các cột là:")
percent_missing = df[cols_category].isnull().mean().sort_values(ascending=False) * 100
percent_missing = percent_missing.apply(lambda x: f"{x:.2f}%")
print(percent_missing)

**Nhận xét:** Các cột phân loại đa số thiếu dữ liệu rất ít, có cột country_name là thiếu 14.16% nhưng cũng không quá cao.

#### Có sự không nhất quán trong các danh mục không? Có các giá trị bất ngờ hoặc bất thường không? 

- Tất cả các cột đều chứa giá trị nan, nên thay thế nan bằng unknown để đồng nhất với các giá trị hiện có.
- Đánh giá từng cột:
    + Cột `location_accuracy`: có tính nhất quán khá tốt, không có giá trị bất thường.
    + Cột `landslide_category`: có sự chồng chéo về ngữ nghĩa, "landslide" là thuật ngữ chung, trong khi "mudslide", "rock_fall" là thuật ngữ cụ thể.
    + Cột `landslide_trigger`: Có quá nhiều danh mục chồng chéo liên quan đến "MƯA".
    + Cột `landslide_size`: có tính nhất quán tốt.
    + Cột `landslide_setting`: có tính nhất quán tương đối ổn.
    + Cột `country_name`: có sự không nhất quán và trùng lặp. Trùng lặp tên "Czechia" và "Czech Republic" là một quốc gia, định dạng lạ "Myanmar [Burma]", dữ liệu chứa cả quốc gia độc lập (United States, China) lẫn vùng lãnh thổ phụ thuộc (Hong Kong, U.S. Virgin Islands, Puerto Rico, Guam, American Samoa).

#### Có các danh mục có rất ít quan sát không? Chúng có nên được gộp nhóm lại không?

- Cột `location_accuracy`: Các giá trị như 100km, 250km có số lượng quan sát rất ít. Có thể gộp 50km, 100km, 250km thành nhóm ">50km".
- Cột `landslide_category`: Các nhóm như lahar, topple, creep, earth_flow, translational_slide, snow_avalanche có số lượng quan sát rất ít. Có thể gộp tất cả các nhóm trên và nhóm unknown vào nhóm other.
- Cột `landslide_trigger`: 
    + Gộp rain, downpour, continuous_rain, flooding, monsoon, tropical_cyclone thành một nhóm lớn là "rain". 
    + Gộp construction, mining, leaking_pipe, dam_embankment_collapse thành một nhóm "anthropogenic".
    + Gộp earthquake, volcano, vibration thành một nhóm "Seismic".
    + Gộp unknown, no_apparent_trigger, snowfall_snowmelt, freeze_thaw thành "other". 
- Cột `landslide_size`: nhóm catastrophic quá nhỏ, có thể gộp vào very_large.
- Cột `landslide_setting`: Có thể gộp above_road và below_road thành "road", gộp các nhóm nhỏ (burned_area, bluff, retaining_wall, above_coast) vào "other".
- Cột `country_name`: Không cần phải gộp.

## 4. Phân tích các cột tọa độ
Xét 2 cột `longitude, latitude`

In [ ]:
plt.figure(figsize=(12, 6))

# Vẽ Scatter Plot: X là Kinh độ, Y là Vĩ độ
# hue='landslide_category': Tô màu theo loại sạt lở (tùy chọn)
sns.scatterplot(
    data=df, 
    x='longitude', 
    y='latitude', 
    alpha=0.5,       
    s=10,            
    edgecolor=None  
)

plt.title('Phân bố vị trí các vụ sạt lở (Bản đồ tọa độ)', fontsize=14)
plt.xlabel('Kinh độ', fontsize=12)
plt.ylabel('Vĩ độ', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Giới hạn trục để loại bỏ các điểm nhiễu (nếu có)
plt.xlim(-180, 180)
plt.ylim(-90, 90)

plt.show()

In [ ]:
print(f"Vĩ độ: min = {df.latitude.min():.2f}, max = {df.latitude.max():.2f}")
print(f"Kinh độ: min = {df.longitude.min():.2f}, max = {df.longitude.max():.2f}")

**Nhận xét:**
- Các vụ sạt lở có phạm vi hầu hết các Châu lục trên thế giới
- Châu Á và Bắc Mỹ có số vụ sạt lở dày đặt hơn các Châu lục còn lại

## 5. Phân tích cột thời gian
Xét cột `event_date`

In [ ]:
df_time = pd.DataFrame()

df_time['year'] = df['event_date'].dt.year
df_time['month'] = df['event_date'].dt.month
df_time['hour'] = df['event_date'].dt.hour
df_time['weekday'] = df['event_date'].dt.dayofweek    # 0 = Monday

df_time.head()

In [ ]:
# Thiết lập kích thước
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1. Xu hướng theo Năm (Year Trend)
sns.countplot(data=df_time, x='year', ax=axes[0, 0])
axes[0, 0].set_title('Number of Landslide Events by Year', fontsize=14)
axes[0, 0].set_xlabel('Year', fontsize=12)
axes[0, 0].set_ylabel('Number of Landslides', fontsize=12)
axes[0, 0].tick_params(axis='x', rotation=45)
for p in axes[0, 0].patches:
    axes[0, 0].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

# 2. Tính mùa vụ theo Tháng (Seasonality)
sns.countplot(data=df_time, x='month', ax=axes[0, 1])
axes[0, 1].set_title('Monthly Distribution of Landslide Events (Seasonality)', fontsize=14)
axes[0, 1].set_xlabel('Month', fontsize=12)
axes[0, 1].set_ylabel('Number of Landslides', fontsize=12)
for p in axes[0, 1].patches:
    axes[0, 1].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

# 3. Phân bố theo Giờ (Hourly)
sns.countplot(data=df_time, x='hour', ax=axes[1, 0], color='skyblue', order=range(0,24))
axes[1, 0].set_title('Hourly Distribution of Landslide Events', fontsize=14)
axes[1, 0].set_xlabel('Hour of Day', fontsize=12)
axes[1, 0].set_ylabel('Number of Landslides', fontsize=12)
for p in axes[1, 0].patches:
    axes[1, 0].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

# 4. Kiểm tra thiên kiến ngày trong tuần (Weekday Bias)
# Map 0->Mon, 6->Sun cho dễ nhìn
weekday_map = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
df_viz = df_time.copy()
df_viz['weekday_name'] = df_viz['weekday'].map(weekday_map)
sns.countplot(data=df_viz, x='weekday_name', ax=axes[1, 1], order=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
axes[1, 1].set_title('Distribution of Landslide Events by Day of Week', fontsize=14)
axes[1, 1].set_xlabel('Day of Week', fontsize=12)
axes[1, 1].set_ylabel('Number of Landslides', fontsize=12)
for p in axes[1, 1].patches:
    axes[1, 1].annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

**Nhận xét:**
- Số lượng vụ sạt lở có xu hướng gia tăng theo thời gian, đặc biệt từ sau năm 2006, và duy trì ở mức cao trong các năm tiếp theo. 
- Phân bố theo tháng cho thấy tính mùa vụ rõ rệt, với số vụ sạt lở tập trung nhiều hơn vào các tháng giữa năm, cho thấy mối liên hệ với các yếu tố thời tiết theo mùa. 
- Phân bố theo giờ trong ngày cho thấy sự tập trung bất thường tại mốc 00:00, nhiều khả năng do thiếu thông tin chính xác về thời điểm xảy ra sự kiện, khiến đặc trưng giờ mang ít ý nghĩa phân tích. 
- Phân bố theo thứ trong tuần tương đối đồng đều, phù hợp với bản chất ngẫu nhiên của hiện tượng sạt lở.

## 6. Phân tích dữ liệu bị thiếu

### Đánh giá tổng quan

In [ ]:
df_cols = pd.DataFrame({
    'Count Missing': df.isna().sum(),
    'Percent Missing': df.isnull().sum() * 100 / df.shape[0]
})

df_cols = df_cols.sort_values(by='Percent Missing', ascending=False)

print("Bảng tóm tắt giá trị thiếu")
df_cols

In [ ]:
missing_cnt = df.isnull().sum().sort_values(ascending=False)
missing_pct = (df.isnull().mean() * 100).sort_values(ascending=False)

# Bar chart: số missing
plt.figure(figsize=(10,6))
ax = missing_cnt[missing_cnt > 0].plot(kind='bar')
plt.title('Number of missing values per column')
plt.ylabel('Missing count')
plt.xlabel('Column')
plt.xticks(rotation=45, ha='right')

# optional: annotate counts trên bar
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x() + p.get_width()/2., p.get_height()),
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
print("Biểu đồ trực quan hóa các mẫu dữ liệu bị thiếu")
plt.show()

**Các giá trị thiếu là ngẫu nhiên hay có quy luật? Có hàng hoặc nhóm cụ thể nào có nhiều giá trị thiếu hơn không?**

- Các giá trị thiếu trong bộ dữ liệu này không phải là ngẫu nhiên. Trong thu thập dữ liệu thiên tai, người nhập liệu thường có xu hướng để trống các ô thương vong nếu không có ai bị thương hoặc chết, thay vì điền số 0.
- Cột injury_count (số người bị thương) là cột thiếu dữ liệu nhiều nhất, lên tới 51.43%. Các cột còn lại đều thiếu dưới 15%, tương đối ít.

### Chiến lược cho từng cột

**Đối với mỗi cột có giá trị thiếu:**
1. Nhóm dữ liệu thiệt hại con người `injury_count, fatality_count`:
- Các giá trị có thể bị thiếu vì người ghi nhận thường để trống thay vì điền số 0 khi không có thương vong. 
- Kế hoạch xử lý: Điền khuyết bằng 0.
2. Nhóm dữ liệu Địa lý/Hành chính `country_name, admin_division_name, admin_division_population, gazeteer_distance, gazeteer_closest_point`:
- Các giá trị có thể bị thiếu vì hệ thống có tọa độ nhưng không tìm thấy tên quốc gia/tỉnh thành tương ứng trong cơ sở dữ liệu.
- Kế hoạch xử lý: Điền khuyết bằng "unknown" hoặc "other" đối với biến có kiểu dữ liệu object, điền khuyết bằng 0 đối với biến số.
3. Nhóm dữ liệu phân loại `landslide_trigger, landslide_setting, landslide_accuracy, landslide_category, landslide_size`:
- Các giá trị có thể bị thiếu vì người quan sát không xác định được tại thời điểm đó.
- Kế hoạch xử lý: Điền khuyết bằng "unknown" hoặc "other".
4. Nhóm mô tả văn bản `event_description, location_description`:
- Các giá trị có thể bị thiếu vì người nhập liệu không viết mô tả chi tiết hoặc không có thông tin bổ sung.
- Kế hoạch xử lý: Điền bằng chuỗi rỗng "" hoặc "No description".

## 7. Mối quan hệ và Tương quan

### Ma trận tương quan cho các biến số và biểu đồ tương quan

In [ ]:
# Lọc dữ liệu
corr_df = df[['fatality_count', 'injury_count', 'admin_division_population', 'gazeteer_distance']].dropna()
# Tính hệ số tương quan
correlation_matrix = corr_df.corr()

plt.figure(figsize=(12, 8))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, vmin=-1, vmax=1)
plt.title('Ma trận tương quan giữa các biến số', fontsize=14)
plt.show()

**Nhận xét:**
- Không có mối tương quan nào giữa các biến số.
- Tất cả hệ số tương quan đều dưới 0.15, cho thấy mối quan hệ yếu, không xảy ra hiện tượng đa cộng tuyến nghiêm trọng.
- Không có mối quan hệ nào thật sự “đáng ngạc nhiên”.

### Bảng chéo

**Tổ hợp quan trọng giữa biến phân loại × biến phân loại**  
1. Xét cặp biến landslide_trigger (Nguyên nhân) và landslide_category (Phân loại sạt lở).

In [ ]:
# Tạo bảng tần suất (Số lượng sự kiện)
ct_counts = pd.crosstab(df['landslide_trigger'], df['landslide_category'])

print("Bảng tần suất")
ct_counts


**Nhận xét:** Nguyên nhân do mưa lớn là yếu tố chủ yếu dẫn đến sạt lở đất, một phần nó cũng gây ra đá lở.

2. Xét cặp biến landslide_trigger (Nguyên nhân) và landslide_size (Quy mô sạt lở).

In [ ]:
print("Bảng tần suất")
pd.crosstab(df['landslide_trigger'], df['landslide_size'])

**Nhận xét:** Nguyên nhân do mưa lớn thường gây sạt lở đất quy mô từ nhỏ đến trung bình 

**Tổ hợp biến số × biến phân loại**  
Xét cặp biến landslide_size (Quy mô sạt lở) và fatality_count (Số người chết)

In [ ]:
df.groupby('landslide_size')['fatality_count'].agg(
    count='count',
    mean='mean',
    median='median'
)

**Nhận xét:**
- Xu hướng cho thấy quy mô sạt lở càng lớn thì số người chết càng cao . 
- Hơn 50% vụ sạt lở có quy mô nhỏ và trung bình thì không gây chết người (trung vị = 0).
- Nhóm quy mô lớn trở đi thì có số lượng người chết càng tăng lên.

## 8. Quan sát ban đầu và Insight

### Tóm tắt

#### Quan sát chính từ quá trình khám phá dữ liệu
- Mưa là nguyên nhân chủ yếu gây ra sạt lở đất.
- Các vụ sạt lở đất được báo cáo thì ở nước Mỹ là nhiều nhất do người thu thập có thể đã thu thập rất nhiều ở Mỹ thay vì ở những quốc gia khác.
- Châu Á và Bắc Mỹ có số vụ sạt lở dày đặt hơn các Châu lục còn lại.
- Số vụ sạt lở có xu hướng tăng dần theo thời gian đặc biệt là từ 2007 trở đi và tập trung vào các tháng như 6, 7, 8 là chủ yếu.

#### Vấn đề về chất lượng dữ liệu 
- Dữ liệu về sạt lở này có chất lượng tương đối ổn.
- Tuy nhiên, có cột injury_count (số người bị thương) là cột thiếu dữ liệu nhiều nhất lên tới 51.43%.
- Dữ liệu lấy từ thực tế nên các đặc trưng đa số đều sẽ có phân phối lệch phải và có nhiều giá trị ngoại lai do trong thực tế có vụ sạt lở rất lớn làm chết nhiều người nhưng hiếm khi xảy ra.

#### Chiến lược cho tiền xử lý dữ liệu

#### Những mẫu thú vị có thể dẫn đến các câu hỏi nghiên cứu

In [ ]:
(df['admin_division_population'] > 1_000_000).sum()